## FASE 2: DATA UNDERSTANDING (Pemahaman Data)

### Blok 2.1: Import Pustaka (Library) dan Muat Data

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
import sys

# Muat data dari file yang diunggah
file_path = 'C:/Users/Reinaldy Hutapea/Videos/SEMESTER 7/SISREK/Kelompok/data-books/train.csv'
df = pd.read_csv(file_path)

print("Data berhasil dimuat.")

Data berhasil dimuat.


### Blok 2.2: Eksplorasi Data Awal (Inspeksi Kualitas)

In [2]:
print("--- 5 Baris Pertama Data (Inspeksi Visual) ---")
print(df.head())

print("\n--- Informasi Teknis Data (Pemeriksaan Kualitas) ---")
df.info()

--- 5 Baris Pertama Data (Inspeksi Visual) ---
   user_id     item_id
0        8  0002005018
1        8  074322678X
2        8  0887841740
3        8  1552041778
4        8  1567407781

--- Informasi Teknis Data (Pemeriksaan Kualitas) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269764 entries, 0 to 269763
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user_id  269764 non-null  int64 
 1   item_id  269764 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.1+ MB


### Blok 2.3: Statistik Deskriptif Data

In [3]:
# Hitung statistik kunci
n_users = df['user_id'].nunique()
n_items = df['item_id'].nunique()
n_interactions = len(df)

# Hitung kepadatan data (seberapa banyak matriks terisi)
sparsity = n_interactions / (n_users * n_items)

print(f"--- Statistik Kunci Data ---")
print(f"Jumlah Total Interaksi: {n_interactions}")
print(f"Jumlah Pengguna Unik:   {n_users}")
print(f"Jumlah Item (Buku) Unik: {n_items}")
print(f"Sparsity Data: {sparsity:.6f} (atau {sparsity*100:.4f}%)")

--- Statistik Kunci Data ---
Jumlah Total Interaksi: 269764
Jumlah Pengguna Unik:   13876
Jumlah Item (Buku) Unik: 123069
Sparsity Data: 0.000158 (atau 0.0158%)


## FASE 3: DATA PREPARATION (Persiapan Data)

### Blok 3.1: Implementasi Metrik Evaluasi (MAP@10)

In [4]:
def apk(actual, predicted, k=10):
    """ Menghitung Average Precision at k (AP@k) untuk satu pengguna. """
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """ Menghitung Mean Average Precision at k (MAP@k) untuk semua pengguna. """
    if not actual or not predicted or len(actual) != len(predicted):
        return 0.0
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted) if a])

print("Fungsi metrik apk() dan mapk() telah dibuat.")

Fungsi metrik apk() dan mapk() telah dibuat.


### Blok 3.2: Pemetaan ID (ID Mapping)

In [5]:
# Membuat ID Kategori (indeks numerik dari 0)
df['user_id_cat'] = df['user_id'].astype("category").cat.codes
df['item_id_cat'] = df['item_id'].astype("category").cat.codes

# Membuat Kamus Penerjemah
# (ID Kategori -> ID Asli) - Untuk menerjemahkan hasil rekomendasi
cat_to_item_id = dict(zip(df['item_id_cat'], df['item_id']))

# (ID Asli -> ID Kategori) - Untuk mencari data di matriks
user_id_to_cat = dict(zip(df['user_id'], df['user_id_cat']))

print("ID kategori dan kamus pemetaan telah dibuat.")
print("\nContoh data setelah pemetaan (kolom baru di kanan):")
print(df.head())

ID kategori dan kamus pemetaan telah dibuat.

Contoh data setelah pemetaan (kolom baru di kanan):
   user_id     item_id  user_id_cat  item_id_cat
0        8  0002005018            0           84
1        8  074322678X            0        84491
2        8  0887841740            0       100281
3        8  1552041778            0       103153
4        8  1567407781            0       104104


### Blok 3.3: Pembagian Data (Train-Validation Split)

In [6]:
# Membagi data (80% Latihan, 20% Validasi)
train_df, validation_df = train_test_split(df, 
                                           test_size=0.2, 
                                           random_state=42)

print(f"Total data: {len(df)}")
print(f"Jumlah data latihan (train): {len(train_df)} ({len(train_df)/len(df)*100:.1f}%)")
print(f"Jumlah data validasi (validation): {len(validation_df)} ({len(validation_df)/len(df)*100:.1f}%)")

Total data: 269764
Jumlah data latihan (train): 215811 (80.0%)
Jumlah data validasi (validation): 53953 (20.0%)


### Blok 3.4: Finalisasi Set Data Evaluasi

In [7]:
# 1. Membuat "Kunci Jawaban" (Ground Truth) dari data validasi
validation_truth = validation_df.groupby('user_id')['item_id'].apply(list).to_dict()
# Daftar pengguna yang akan kita uji
validation_users = list(validation_truth.keys())

# 2. Membuat "Daftar Filter" (Seen Items) dari data latihan
# Menggunakan ID Asli (String) untuk filtering di Fase Evaluasi
train_items_by_user_str = train_df.groupby('user_id')['item_id'].apply(set).to_dict()
# Menggunakan ID Kategori (Angka) untuk proses Modeling
train_items_by_user_cat = train_df.groupby('user_id_cat')['item_id_cat'].apply(list).to_dict()

print(f"Data evaluasi telah siap.")
print(f"Jumlah pengguna yang akan diuji (di set validasi): {len(validation_users)}")

Data evaluasi telah siap.
Jumlah pengguna yang akan diuji (di set validasi): 11443


## FASE 4: MODELING (Pemodelan)

### Blok 4.1: Pelatihan Model A (Baseline): Popularitas

In [8]:
print("--- [MODEL A] Melatih Model Dasar (Popularitas) ---")

# "Pelatihan" = Menghitung item paling populer HANYA dari data latihan
popularity_scores = train_df['item_id'].value_counts()

# Ambil 100 item terpopuler sebagai "model" kita
top_popular_items = popularity_scores.head(100).index.tolist()

print("Model Popularitas telah 'terlatih'.")
print("Top 5 item terpopuler:")
print(top_popular_items[:5])

--- [MODEL A] Melatih Model Dasar (Popularitas) ---
Model Popularitas telah 'terlatih'.
Top 5 item terpopuler:
['0316666343', '0385504209', '0312195516', '0142001740', '059035342X']


### Blok 4.2: Pelatihan Model B (Canggih): Persiapan Matriks IBCF

In [9]:
print("--- [MODEL B] Melatih Model Canggih (IBCF) ---")
print("Langkah 1: Membuat Matriks Sparse User-Item dari data latihan...")

# Ambil jumlah total pengguna/item dari data asli (setelah pemetaan)
num_users = df['user_id_cat'].nunique()
num_items = df['item_id_cat'].nunique()

# Buat matriks (user, item) dari data latihan
user_item_matrix = csr_matrix((np.ones(len(train_df)), # data (semua bernilai 1)
                               (train_df['user_id_cat'], train_df['item_id_cat'])), # (baris, kolom)
                              shape=(num_users, num_items))

print(f"Bentuk matriks User-Item (CSR): {user_item_matrix.shape}")

--- [MODEL B] Melatih Model Canggih (IBCF) ---
Langkah 1: Membuat Matriks Sparse User-Item dari data latihan...
Bentuk matriks User-Item (CSR): (13876, 123069)


### Blok 4.3: Pelatihan Model B (Canggih): Perhitungan Kesamaan IBCF

In [10]:
print("Langkah 2: Menghitung Matriks Kesamaan Item (Inti Model)...")

# Transposisi matriks ke (Item, User)
item_user_matrix = user_item_matrix.T.tocsr()

# Hitung Cosine Similarity antar item
item_similarity_matrix = cosine_similarity(item_user_matrix, dense_output=False)
item_similarity_matrix = item_similarity_matrix.tocsr() # Ubah ke CSR untuk pencarian cepat

print(f"Bentuk matriks Kesamaan Item (Model Terlatih): {item_similarity_matrix.shape}")

Langkah 2: Menghitung Matriks Kesamaan Item (Inti Model)...
Bentuk matriks Kesamaan Item (Model Terlatih): (123069, 123069)


## FASE 5: EVALUATION (Evaluasi)

### Blok 5.1: Evaluasi Model A (Popularitas)

In [11]:
print("--- [EVALUASI MODEL A] Menghitung Skor MAP@10 Popularitas ---")

predicted_list_popular = []
actual_list_popular = []

for user in validation_users:
    # 1. Ambil daftar item yang sudah dilihat (untuk filtering)
    seen_items = train_items_by_user_str.get(user, set())

    # 2. Buat rekomendasi: item populer yang BELUM dilihat
    recs = [item for item in top_popular_items if item not in seen_items]

    # 3. Ambil top 10
    top_10_recs = recs[:10]

    # 4. Kumpulkan prediksi dan kunci jawaban
    predicted_list_popular.append(top_10_recs)
    actual_list_popular.append(validation_truth[user])

# 5. Hitung skor akhir
mapk_score_popular = mapk(actual_list_popular, predicted_list_popular, k=10)

print(f"--- SKOR AKHIR MODEL DASAR (POPULARITAS) ---")
print(f"SKOR MAP@10: {mapk_score_popular:.6f}")

--- [EVALUASI MODEL A] Menghitung Skor MAP@10 Popularitas ---
--- SKOR AKHIR MODEL DASAR (POPULARITAS) ---
SKOR MAP@10: 0.004362


### Blok 5.2: Evaluasi Model B (Canggih - IBCF)

In [12]:
print("--- [EVALUASI MODEL B] Menghitung Skor MAP@10 IBCF ---")
import time
start_time = time.time()

# --- Fungsi Bantu untuk Rekomendasi IBCF ---
def recommend_ibcf(user_cat_id, train_items_user_cat, similarity_matrix):
    """ Menghasilkan skor mentah untuk semua item berdasarkan riwayat pengguna. """
    seen_items_cat = train_items_user_cat.get(user_cat_id, [])
    if not seen_items_cat:
        return np.array([])
    # Ambil vektor kesamaan dan jumlahkan
    similar_scores = similarity_matrix[seen_items_cat, :].sum(axis=0)
    return similar_scores.A1 # .A1 mengubahnya jadi array 1D
# --- ------------------------------------ ---

predicted_list_ibcf = []
actual_list_ibcf = []

for i, user_id_str in enumerate(validation_users):
    user_cat_id = user_id_to_cat.get(user_id_str)
    if user_cat_id is None: # Jika pengguna tidak ada di set latihan
        predicted_list_ibcf.append([])
        actual_list_ibcf.append(validation_truth.get(user_id_str, []))
        continue

    # 1. Dapatkan skor rekomendasi personal
    raw_scores = recommend_ibcf(user_cat_id, train_items_by_user_cat, item_similarity_matrix)

    # 2. Urutkan skor (indeks item)
    top_indices = np.argsort(raw_scores)[::-1]

    # 3. Filter dan ambil Top 10
    top_10_recs_str = []
    seen_items_str = train_items_by_user_str.get(user_id_str, set())

    for item_cat in top_indices:
        item_id_str = cat_to_item_id[item_cat] # Terjemahkan kembali
        if item_id_str not in seen_items_str: # Lakukan filtering
            top_10_recs_str.append(item_id_str)
        if len(top_10_recs_str) >= 10:
            break

    # 4. Kumpulkan hasil
    predicted_list_ibcf.append(top_10_recs_str)
    actual_list_ibcf.append(validation_truth[user_id_str])

    if (i + 1) % 2000 == 0:
        print(f"Selesai memproses {i+1} / {len(validation_users)} pengguna...")

end_time = time.time()
print(f"Proses evaluasi IBCF selesai dalam {end_time - start_time:.2f} detik.")

# 5. Hitung skor akhir
mapk_score_ibcf = mapk(actual_list_ibcf, predicted_list_ibcf, k=10)

print(f"\n--- SKOR AKHIR MODEL CANGGIH (IBCF) ---")
print(f"SKOR MAP@10: {mapk_score_ibcf:.6f}")

--- [EVALUASI MODEL B] Menghitung Skor MAP@10 IBCF ---
Selesai memproses 2000 / 11443 pengguna...
Selesai memproses 4000 / 11443 pengguna...
Selesai memproses 6000 / 11443 pengguna...
Selesai memproses 8000 / 11443 pengguna...
Selesai memproses 10000 / 11443 pengguna...
Proses evaluasi IBCF selesai dalam 41.79 detik.

--- SKOR AKHIR MODEL CANGGIH (IBCF) ---
SKOR MAP@10: 0.007221


## FASE 6: DEPLOYMENT (Kesimpulan Proyek)

### Blok 6.1: Perbandingan Hasil dan Kesimpulan Akhir (Cell Teks)

In [13]:
print("--- [FASE 6] KESIMPULAN AKHIR PROYEK ---")
print(f"Skor Model Dasar (Popularitas): MAP@10 = {mapk_score_popular:.6f}")
print(f"Skor Model Canggih (IBCF):     MAP@10 = {mapk_score_ibcf:.6f}")

# --- Analisis Hasil ---
if mapk_score_ibcf > mapk_score_popular:
    peningkatan = ((mapk_score_ibcf / mapk_score_popular) - 1) * 100
    print(f"\nModel IBCF {peningkatan:.2f}% lebih baik daripada model Popularitas.")
    print("KESIMPULAN: Target proyek TERCAPAI. Model canggih terbukti lebih baik.")
else:
    print("\nKESIMPULAN: Target proyek TIDAK TERCAPAI. Model canggih tidak lebih baik.")

--- [FASE 6] KESIMPULAN AKHIR PROYEK ---
Skor Model Dasar (Popularitas): MAP@10 = 0.004362
Skor Model Canggih (IBCF):     MAP@10 = 0.007221

Model IBCF 65.53% lebih baik daripada model Popularitas.
KESIMPULAN: Target proyek TERCAPAI. Model canggih terbukti lebih baik.
